In [28]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.metrics import accuracy_score,f1_score,precision_score,roc_auc_score

ImportError: cannot import name 'roc_aoc_score' from 'sklearn.metrics' (/opt/conda/lib/python3.10/site-packages/sklearn/metrics/__init__.py)

In [4]:
df=pd.read_csv("/kaggle/input/tertiary-mushroom-1-million-more-mushrooms/one_million_mushrooms.csv",delimiter=';')

In [5]:
df.isnull().sum()

class                        0
cap-diameter                 0
cap-shape                    0
cap-surface             240000
cap-color                    0
does-bruise-or-bleed         0
gill-attachment         168000
gill-spacing            426000
gill-color                   0
stem-height                  0
stem-width                   0
stem-root               876000
stem-surface            648000
stem-color                   0
veil-type               984000
veil-color              912000
has-ring                     0
ring-type                42000
spore-print-color       930000
habitat                      0
season                       0
dtype: int64

In [6]:
new_df=df.drop(columns=["cap-surface","gill-attachment","gill-spacing","stem-root","stem-surface","veil-type","veil-color","spore-print-color","ring-type"])

In [7]:
new_df.isnull().sum()

class                   0
cap-diameter            0
cap-shape               0
cap-color               0
does-bruise-or-bleed    0
gill-color              0
stem-height             0
stem-width              0
stem-color              0
has-ring                0
habitat                 0
season                  0
dtype: int64

In [8]:
new_df

,class,cap-diameter,cap-shape,cap-color,does-bruise-or-bleed,gill-color,stem-height,stem-width,stem-color,has-ring,habitat,season
0,p,5.68,c,w,t,r,6.73,12.79,w,f,g,u
1,e,5.62,f,n,f,w,6.10,9.50,n,f,m,u
2,p,17.29,x,n,f,y,6.80,52.87,n,f,d,a
3,p,6.16,b,n,f,w,9.33,11.62,w,t,d,s
4,p,4.46,o,n,f,w,0.00,0.00,f,f,d,u
...,...,...,...,...,...,...,...,...,...,...,...,...
1037995,p,7.03,f,e,t,y,5.83,9.30,e,f,d,u
1037996,e,4.92,x,w,f,n,7.02,4.27,w,f,d,a
1037997,p,8.68,x,e,t,w,8.98,9.89,n,f,d,a
1037998,p,8.08,f,e,t,w,6.32,8.62,e,f,d,a


In [9]:
one_hot_columns=OneHotEncoder(sparse_output=False)
column_hot=one_hot_columns.fit_transform(df[["cap-shape","cap-color","does-bruise-or-bleed","gill-color","stem-color","has-ring","habitat","season"]])
column_hot=pd.DataFrame(column_hot,columns=one_hot_columns.get_feature_names_out())

In [10]:
column_hot["cap-diameter"]=new_df["cap-diameter"]
column_hot["stem-height"]=new_df["stem-height"]
column_hot["stem-width"]=new_df["stem-width"]
# column_hot["class_encoder"]=new_df["class_encoder"]

In [11]:
column_hot

,cap-shape_b,cap-shape_c,cap-shape_f,cap-shape_o,cap-shape_p,cap-shape_s,cap-shape_x,cap-color_b,cap-color_e,cap-color_g,cap-color_k,cap-color_l,cap-color_n,cap-color_o,cap-color_p,cap-color_r,cap-color_u,cap-color_w,cap-color_y,does-bruise-or-bleed_f,does-bruise-or-bleed_t,gill-color_b,gill-color_e,gill-color_f,gill-color_g,gill-color_k,gill-color_n,gill-color_o,gill-color_p,gill-color_r,gill-color_u,gill-color_w,gill-color_y,stem-color_b,stem-color_e,stem-color_f,stem-color_g,stem-color_k,stem-color_l,stem-color_n,stem-color_o,stem-color_p,stem-color_r,stem-color_u,stem-color_w,stem-color_y,has-ring_f,has-ring_t,habitat_d,habitat_g,habitat_h,habitat_l,habitat_m,habitat_p,habitat_u,habitat_w,season_a,season_s,season_u,season_w,cap-diameter,stem-height,stem-width
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,5.68,6.73,12.79
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,5.62,6.10,9.50
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,17.29,6.80,52.87
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6.16,9.33,11.62
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4.46,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1037995,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,7.03,5.83,9.30
1037996,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4.92,7.02,4.27
1037997,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,8.68,8.98,9.89
1037998,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,8.08,6.32,8.62


In [12]:
one_hot=OneHotEncoder(sparse_output=False)
label_encoder=LabelEncoder()
new_df["class_encoder"]=label_encoder.fit_transform(new_df["class"])
one=one_hot.fit_transform(new_df[["class"]])
one=pd.DataFrame(one,columns=one_hot.get_feature_names_out())

In [13]:
# X=column_hot
# Y=n

In [14]:
X_train,X_test,Y_train,Y_test=train_test_split(column_hot,one,test_size=0.2,random_state=42)

In [15]:
# random_classif=RandomForestClassifier()
# random_classif.fit(X_train,Y_train)
# random_classif.score(X_test,Y_test)

# # Deep learning

In [21]:
from keras.layers import Dense,Flatten,BatchNormalization,TimeDistributed,Input,Dropout
from keras.models import Model
from keras.initializers import HeNormal
from keras.regularizers import l2
from keras.callbacks import EarlyStopping


In [22]:
early_stopping=EarlyStopping(
monitor='val_loss',
patience=5,
verbose=1
    
)

In [23]:
X_train.shape

(830400, 63)

In [26]:
inputs=Input(shape=(63,))
D=Dense(32,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(inputs)
D=Dense(32,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
D=Dropout(0.2)(D)
D=Dense(64,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
D=Dense(64,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
D=Dropout(0.5)(D)
D=Dense(128,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(inputs)
D=Dense(128,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
D=Dropout(0.2)(D)
D=Dense(256,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
D=Dense(256,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
outputs=Dense(2,activation="sigmoid")(D)
model=Model(inputs,outputs)
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=["accuracy"])

In [27]:
model.fit(X_train,Y_train,epochs=32,batch_size=32,validation_data=(X_test,Y_test),callbacks=[early_stopping])

Epoch 1/32
25950/25950 ━━━━━━━━━━━━━━━━━━━━ 82s 3ms/step - accuracy: 0.8794 - loss: 0.8032 - val_accuracy: 0.9838 - val_loss: 0.1736
Epoch 2/32
25950/25950 ━━━━━━━━━━━━━━━━━━━━ 80s 3ms/step - accuracy: 0.9733 - loss: 0.1881 - val_accuracy: 0.9771 - val_loss: 0.1683
Epoch 3/32
25950/25950 ━━━━━━━━━━━━━━━━━━━━ 78s 3ms/step - accuracy: 0.9738 - loss: 0.1744 - val_accuracy: 0.9869 - val_loss: 0.1425
Epoch 4/32
25950/25950 ━━━━━━━━━━━━━━━━━━━━ 79s 3ms/step - accuracy: 0.9739 - loss: 0.1688 - val_accuracy: 0.9748 - val_loss: 0.1640
Epoch 5/32
25950/25950 ━━━━━━━━━━━━━━━━━━━━ 80s 3ms/step - accuracy: 0.9744 - loss: 0.1661 - val_accuracy: 0.9697 - val_loss: 0.1679
Epoch 6/32
25950/25950 ━━━━━━━━━━━━━━━━━━━━ 79s 3ms/step - accuracy: 0.9745 - loss: 0.1633 - val_accuracy: 0.9499 - val_loss: 0.2189
Epoch 7/32
25950/25950 ━━━━━━━━━━━━━━━━━━━━ 80s 3ms/step - accuracy: 0.9744 - loss: 0.1627 - val_accuracy: 0.9800 - val_loss: 0.1548
Epoch 8/32
25950/25950 ━━━━━━━━━━━━━━━━━━━━ 80s 3ms/step - accuracy: 

In [29]:
loss,accuracy=model.evaluate(X_test,Y_test)

6488/6488 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.9792 - loss: 0.1474
